In [1]:
import pandas as pd
import numpy as np

In [2]:
from preprocessing_utils import process_sat_data

In [3]:
data_train = pd.read_csv('../data/train.csv')
data_test = pd.read_csv('../data/track1/test.csv')

In [4]:
process_sat_data(data_train, data_test, scale=10000)

# Processing

In [5]:
cols_sim = ['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim']
cols_real = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

In [6]:
train_sat_ids = data_train['sat_id'].values
test_sat_ids = data_test['sat_id'].values

In [7]:
train_sim = data_train.loc[:, cols_sim].values
test_sim = data_test.loc[:, cols_sim].values
train_real = data_train.loc[:, cols_real].values

In [8]:
def extract_windows(data, batch_size):
    windows = []
    for index in range(data.shape[0] - batch_size + 1):
        window = data[index : (index+batch_size), :]
        windows.append(window)
    return np.array(windows)

In [9]:
def get_slices(data, sat_ids, batch_size):
    sat_windows = {}
    for sat_id in range(10):
        sat_data = data[sat_ids == sat_id]
        if (sat_data.shape[0] == 0):
            continue
        sat_windows[sat_id] = extract_windows(sat_data, batch_size)
    return sat_windows

In [10]:
batch_size = 4

In [11]:
train_sim_slices = get_slices(train_sim, train_sat_ids, batch_size)
test_sim_slices = get_slices(test_sim, test_sat_ids, batch_size)
train_real_slices = get_slices(train_sim, train_sat_ids, batch_size)

# Model

In [12]:
import torch
import torch.utils.data as data

In [13]:
class SatDataset(data.Dataset):
    
    def __init__(self, sat_id, data_dict_sim, data_dict_real):
        dat_x = data_dict_sim[sat_id];
        labels_x = data_dict_real[sat_id];
        
    def __getitem__(self, index):
        return dat_x[index], labels_x[index]
    
    def __len__(self):
        return len(data_x)

In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        return output

In [4]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [6]:
class ConcatenationEmbedding(nn.Module):
    
    def forward(self, x):
        N, L, S = x.shape
        concat_x = torch.reshape(x, (N, L*S))
        return concat_x
        

In [10]:
ntokens = 4 # the size of vocabulary
emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout)

In [11]:
lr = 5.0 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    ntokens = len(TEXT.vocab.stoi)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = 200
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.2f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // bptt, scheduler.get_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()